In [153]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [154]:
selected_columns = [
    'Street Code1',
    'Street Code2',
    'Street Code3',
    # 'Violation Location',
    'Violation Post Code',
    'Intersecting Street',
    'Violation Precinct',
    'Violation County',
    'Vehicle Body Type',
    'Issue Date',
    # 'Date First Observed',
    'Violation Time',
    'Days Parking In Effect',
    'From Hours In Effect',
    'To Hours In Effect',
    'Violation Code',
]

'''
Genel olarak, "county" (ilçe) daha büyük bir alanı temsil eder. Bir "county", genellikle belirli bir eyaletin sınırları içinde bulunan büyük bir coğrafi bölgeyi kapsar. Bir eyalette birden çok "county" bulunabilir ve her biri kendi yerel yönetimine sahiptir.

Öte yandan, "precinct" (seçim bölgesi) daha küçük bir coğrafi bölgeyi ifade eder. Bu, genellikle seçmenlerin oy kullandığı belirli bir mahalle veya köyü temsil eder. Seçim bölgesi, seçimlerde kullanılan ve oy verme işlemini düzenleyen bir kavramdır.
'''

n_rows_to_read = 350000
file_path = 'Parking_Violations_2022.csv'
df = pd.read_csv(file_path, usecols=selected_columns, nrows=n_rows_to_read) # nrows=n_rows_to_read
df

C:\Users\YSK\AppData\Local\Temp\ipykernel_11308\1023093742.py:28: DtypeWarning: Columns (30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=selected_columns, nrows=n_rows_to_read) # nrows=n_rows_to_read


,Issue Date,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Intersecting Street,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Violation Post Code
0,06/25/2021,40,VAN,63430,69230,13490,88,0130A,K,NaN,BBBBBBB,ALL,ALL,NaN
1,06/25/2021,20,SUBN,13490,40404,40404,88,0225A,K,NaN,BBBBBBB,ALL,ALL,NaN
2,06/17/2021,98,SDN,79430,47130,11750,88,0809P,K,NaN,BBBBBBB,ALL,ALL,NaN
3,06/16/2021,98,SDN,53130,23230,23930,83,0605P,K,NaN,BBBBBBB,ALL,ALL,NaN
4,07/04/2021,40,TAXI,81030,23930,40030,83,1058P,K,NaN,BBBBBBB,ALL,ALL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349995,07/27/2021,36,SUBN,0,0,0,0,1204P,QN,1ST ST,NaN,NaN,NaN,NaN
349996,07/27/2021,36,SUBN,0,0,0,0,1204P,BK,Y @ BILLS PL,NaN,NaN,NaN,NaN
349997,07/27/2021,36,SUBN,0,0,0,0,1204P,ST,ONIA AVE,NaN,NaN,NaN,NaN
349998,07/27/2021,36,SUBN,0,0,0,0,1204P,QN,TH ST,NaN,NaN,NaN,NaN


In [172]:
df['Intersecting Street'].isna().sum()

53078

In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350000 entries, 0 to 349999
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Issue Date              350000 non-null  object 
 1   Violation Code          350000 non-null  int64  
 2   Vehicle Body Type       347859 non-null  object 
 3   Street Code1            350000 non-null  int64  
 4   Street Code2            350000 non-null  int64  
 5   Street Code3            350000 non-null  int64  
 6   Violation Precinct      350000 non-null  int64  
 7   Violation Time          349991 non-null  object 
 8   Violation County        348269 non-null  object 
 9   Intersecting Street     296922 non-null  object 
 10  Days Parking In Effect  34361 non-null   object 
 11  From Hours In Effect    39343 non-null   object 
 12  To Hours In Effect      39342 non-null   object 
 13  Violation Post Code     0 non-null       float64
dtypes: float64(1), int64

In [157]:
df['Violation Time']

0         0130A
1         0225A
2         0809P
3         0605P
4         1058P
          ...  
349995    1204P
349996    1204P
349997    1204P
349998    1204P
349999    1204P
Name: Violation Time, Length: 350000, dtype: object

In [158]:
df_1 = df.copy()

In [159]:
# import sweetviz as sv
# analyze_report = sv.analyze(df_1)
# analyze_report.show_html('analyze.html', open_browser=False)

In [160]:
df_1['Violation Date'] = pd.to_datetime(df_1['Issue Date'])
df_1['Violation Date (Month)'] = df_1['Violation Date'].dt.month
df_1['Violation Date (Day)'] = df_1['Violation Date'].dt.day
df_1['Violation Date (Year)'] = df_1['Violation Date'].dt.year
df_1 = df_1.drop(columns=['Issue Date'])

In [161]:
df_1

,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Intersecting Street,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Violation Post Code,Violation Date,Violation Date (Month),Violation Date (Day),Violation Date (Year)
0,40,VAN,63430,69230,13490,88,0130A,K,NaN,BBBBBBB,ALL,ALL,NaN,2021-06-25,6,25,2021
1,20,SUBN,13490,40404,40404,88,0225A,K,NaN,BBBBBBB,ALL,ALL,NaN,2021-06-25,6,25,2021
2,98,SDN,79430,47130,11750,88,0809P,K,NaN,BBBBBBB,ALL,ALL,NaN,2021-06-17,6,17,2021
3,98,SDN,53130,23230,23930,83,0605P,K,NaN,BBBBBBB,ALL,ALL,NaN,2021-06-16,6,16,2021
4,40,TAXI,81030,23930,40030,83,1058P,K,NaN,BBBBBBB,ALL,ALL,NaN,2021-07-04,7,4,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349995,36,SUBN,0,0,0,0,1204P,QN,1ST ST,NaN,NaN,NaN,NaN,2021-07-27,7,27,2021
349996,36,SUBN,0,0,0,0,1204P,BK,Y @ BILLS PL,NaN,NaN,NaN,NaN,2021-07-27,7,27,2021
349997,36,SUBN,0,0,0,0,1204P,ST,ONIA AVE,NaN,NaN,NaN,NaN,2021-07-27,7,27,2021
349998,36,SUBN,0,0,0,0,1204P,QN,TH ST,NaN,NaN,NaN,NaN,2021-07-27,7,27,2021


In [162]:
def convert_to_24hr_format(time_str):
    if '.' in time_str:
        return None
    if time_str[-1] == 'P':
        return '{:02d}:{:02d}'.format((int(time_str[:2]) + 12) % 24, int(time_str[2:4]))
    elif time_str[-1] == 'A':
        return '{:02d}:{:02d}'.format(int(time_str[:2]) % 12, int(time_str[2:4]))
    else:
        return None

df_1['Violation Time'] = df_1['Violation Time'].astype(str)
df_1.loc[:, 'Violation Time'] = df_1['Violation Time'].apply(convert_to_24hr_format)
df_1 = df_1.dropna(subset=['Violation Time'])

In [163]:
df_1.loc[:, 'Violation Time'] = pd.to_datetime(df_1['Violation Time'], format='%H:%M', errors='coerce').dt.time

C:\Users\YSK\AppData\Local\Temp\ipykernel_11308\163225079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1.loc[:, 'Violation Time'] = pd.to_datetime(df_1['Violation Time'], format='%H:%M', errors='coerce').dt.time


In [164]:
df_1['Violation Time']

0         01:30:00
1         02:25:00
2         20:09:00
3         18:05:00
4         22:58:00
            ...   
349995    00:04:00
349996    00:04:00
349997    00:04:00
349998    00:04:00
349999    00:04:00
Name: Violation Time, Length: 349983, dtype: object

In [165]:
df_1

,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Intersecting Street,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Violation Post Code,Violation Date,Violation Date (Month),Violation Date (Day),Violation Date (Year)
0,40,VAN,63430,69230,13490,88,01:30:00,K,NaN,BBBBBBB,ALL,ALL,NaN,2021-06-25,6,25,2021
1,20,SUBN,13490,40404,40404,88,02:25:00,K,NaN,BBBBBBB,ALL,ALL,NaN,2021-06-25,6,25,2021
2,98,SDN,79430,47130,11750,88,20:09:00,K,NaN,BBBBBBB,ALL,ALL,NaN,2021-06-17,6,17,2021
3,98,SDN,53130,23230,23930,83,18:05:00,K,NaN,BBBBBBB,ALL,ALL,NaN,2021-06-16,6,16,2021
4,40,TAXI,81030,23930,40030,83,22:58:00,K,NaN,BBBBBBB,ALL,ALL,NaN,2021-07-04,7,4,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349995,36,SUBN,0,0,0,0,00:04:00,QN,1ST ST,NaN,NaN,NaN,NaN,2021-07-27,7,27,2021
349996,36,SUBN,0,0,0,0,00:04:00,BK,Y @ BILLS PL,NaN,NaN,NaN,NaN,2021-07-27,7,27,2021
349997,36,SUBN,0,0,0,0,00:04:00,ST,ONIA AVE,NaN,NaN,NaN,NaN,2021-07-27,7,27,2021
349998,36,SUBN,0,0,0,0,00:04:00,QN,TH ST,NaN,NaN,NaN,NaN,2021-07-27,7,27,2021


In [166]:
df_1.sort_values(by=['Violation Date', 'Violation Time'], inplace=True)
df_1

C:\Users\YSK\AppData\Local\Temp\ipykernel_11308\735695388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1.sort_values(by=['Violation Date', 'Violation Time'], inplace=True)


,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Intersecting Street,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Violation Post Code,Violation Date,Violation Date (Month),Violation Date (Day),Violation Date (Year)
6647,20,SUBN,72720,27180,29220,46,00:04:00,BX,NaN,BBBBBBB,ALL,ALL,NaN,2000-06-27,6,27,2000
3082,78,NaN,0,40404,40404,105,22:00:00,Q,NaN,BBBBBBB,ALL,ALL,NaN,2000-06-28,6,28,2000
5337,35,VAN,10610,40404,40404,14,00:52:00,NY,NaN,BBBBBBB,ALL,ALL,NaN,2000-07-15,7,15,2000
15310,67,SDN,68390,48290,54090,104,09:10:00,NaN,NaN,BBBBBBB,ALL,ALL,NaN,2010-07-10,7,10,2010
1635,98,VAN,19990,17540,17740,106,01:05:00,Q,NaN,BBBBBBB,ALL,ALL,NaN,2010-07-21,7,21,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7140,14,SUBN,62200,33490,0,100,15:31:00,Q,BEACH 100 ST,BBBBBBB,ALL,ALL,NaN,2022-07-20,7,20,2022
3717,74,MOPE,31250,27950,47530,121,04:30:00,R,NaN,BBBBBBB,ALL,ALL,NaN,2024-07-24,7,24,2024
3730,74,MOPE,31250,27950,47530,121,04:30:00,R,NaN,BBBBBBB,ALL,ALL,NaN,2024-07-24,7,24,2024
9132,67,SDN,0,0,0,71,08:51:00,K,S/O STERLING ST,BBBBBBB,ALL,ALL,NaN,2029-06-29,6,29,2029


In [167]:
start_date = pd.to_datetime('2021-07-01')
end_date = pd.to_datetime('2022-06-30')

In [168]:
df_1 = df_1[(df_1['Violation Date'] >= start_date) & (df_1['Violation Date'] <= end_date)]
df_1

,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Intersecting Street,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Violation Post Code,Violation Date,Violation Date (Month),Violation Date (Day),Violation Date (Year)
19777,21,4DSD,0,0,0,107,00:00:00,Qns,RK DRIVE EAST,Y,0830A,1000A,NaN,2021-07-01,7,1,2021
22002,0,VAN,0,0,0,0,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-01,7,1,2021
23604,0,SUBN,0,0,0,0,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-01,7,1,2021
26384,21,4DSD,8120,25860,25890,40,00:00:00,Bronx,NaN,Y Y,1130A,0100P,NaN,2021-07-01,7,1,2021
28018,21,SUBN,0,0,0,78,00:00:00,Kings,ET,Y,0930A,1101A,NaN,2021-07-01,7,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14316,20,SDN,13820,0,0,84,20:12:00,K,PIER 5 LOOP,BBBBBBB,ALL,ALL,NaN,2021-08-16,8,16,2021
10511,41,SUBN,32560,49130,13430,81,08:45:00,K,NaN,YBBYBBB,0800A,0930A,NaN,2021-08-17,8,17,2021
10081,20,SUBN,0,30140,0,101,17:08:00,Q,B 17 ST,BBBBBBB,ALL,ALL,NaN,2021-08-31,8,31,2021
10496,41,SUBN,0,0,0,81,08:55:00,K,HERKIMER,YBBYBBB,0800A,0930A,NaN,2021-09-24,9,24,2021


In [169]:
df_1['Violation Hour'] = df_1['Violation Time'].apply(lambda x: (x.hour % 24)).astype(int)
df_1['Violation Hour']

C:\Users\YSK\AppData\Local\Temp\ipykernel_11308\1034078261.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['Violation Hour'] = df_1['Violation Time'].apply(lambda x: (x.hour % 24)).astype(int)


19777     0
22002     0
23604     0
26384     0
28018     0
         ..
14316    20
10511     8
10081    17
10496     8
14226    18
Name: Violation Hour, Length: 297151, dtype: int32

In [170]:
from datetime import datetime

def calculate_location_violation_rate(df, location, time_range):
    start_time = datetime.strptime(time_range[0], "%H:%M").time()
    end_time = datetime.strptime(time_range[1], "%H:%M").time()

    # 'NaT' değerleri kontrol et
    filtered_data = df[(df['Street Code2'] == location) & (~df['Violation Time'].isna()) &
                       (df['Violation Time'].apply(lambda x: not pd.isna(x) and x == x and start_time <= x <= end_time))]

    print(filtered_data)
    # Belirli bir konumun toplam ihlal sayısı
    total_location_violations = len(df[(df['Street Code2'] == location) & (df['Violation Code'] > 0)])

    if total_location_violations == 0:
        return 0

    # Belirli bir konum ve saat aralığındaki ihlal sayısı
    location_violations = len(filtered_data[filtered_data['Violation Code'] > 0])

    # Location Violation Rate hesapla
    location_violation_rate = (location_violations / total_location_violations) * 100

    return location_violation_rate

location = '59520'
time_range = ('08:00', '12:00')
rate = calculate_location_violation_rate(df_1, location, time_range)
print(f'Location Violation Rate for {location} between {time_range[0]} and {time_range[1]}: {rate:.2f}%')

Empty DataFrame
Columns: [Violation Code, Vehicle Body Type, Street Code1, Street Code2, Street Code3, Violation Precinct, Violation Time, Violation County, Intersecting Street, Days Parking In Effect, From Hours In Effect, To Hours In Effect, Violation Post Code, Violation Date, Violation Date (Month), Violation Date (Day), Violation Date (Year), Violation Hour]
Index: []
Location Violation Rate for 59520 between 08:00 and 12:00: 0.00%


In [171]:
def calculate_violation_rate(row, df):
    location = row['Street Code2']
    hour = row['Violation Hour']

    # Belirli bir konum ve saat aralığındaki veriyi filtrele
    filtered_data = df[(df['Street Code2'] == location) & (df['Violation Time'].between(hour, hour+2))]

    # Belirli bir konumun toplam ihlal sayısı
    total_location_violations = len(df[(df['Street Code2'] == location) & (df['Violation Code'] > 0)])

    # Belirli bir konum ve saat aralığındaki ihlal sayısı
    location_violations = len(filtered_data[filtered_data['Violation Code'] > 0])

    # Eğer toplam ihlal sayısı 0 ise sıfıra bölme hatasını önlemek için kontrol ekle
    if total_location_violations == 0:
        return 0

    # Location Violation Rate hesapla
    location_violation_rate = (location_violations / total_location_violations) * 100

    return location_violation_rate

# Violation Rate sütununu DataFrame'e ekle
df['Violation Rate'] = df_1.apply(lambda row: calculate_violation_rate(row, df_1), axis=1)

TypeError: '>=' not supported between instances of 'datetime.time' and 'int'